In [1]:
!python3 -m pip install --upgrade pip

In [1]:
!pip install requests
!pip install os
!pip install pandas

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [3]:
import pandas as pd

# Sample DataFrame creation


df = pd.read_csv("CCEE - colunas.csv", sep=";")

# Converting words in the specified column to lowercase
df['tabelas'] = df['tabelas'].str.lower()

# Display the updated DataFrame
print(df)

                              tabelas  Unnamed: 1  Unnamed: 2  Unnamed: 3
0                             acl_eol         NaN         NaN         NaN
1           agente_qtd_contabilizacao         NaN         NaN         NaN
2    alivio_penalidade_distrib_mensal         NaN         NaN         NaN
3                        biomassa_acl         NaN         NaN         NaN
4              biomassa_fonte_geracao         NaN         NaN         NaN
..                                ...         ...         ...         ...
138          sumario_mensal_resultado         NaN         NaN         NaN
139    usina_cotista_ceg_distribuidor         NaN         NaN         NaN
140  usina_cotista_liquidacao_parcela         NaN         NaN         NaN
141             usina_cotista_nuclear         NaN         NaN         NaN
142               varejista_subclasse         NaN         NaN         NaN

[143 rows x 4 columns]


In [4]:
df.head(144)

,tabelas,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,acl_eol,NaN,NaN,NaN
1,agente_qtd_contabilizacao,NaN,NaN,NaN
2,alivio_penalidade_distrib_mensal,NaN,NaN,NaN
3,biomassa_acl,NaN,NaN,NaN
4,biomassa_fonte_geracao,NaN,NaN,NaN
...,...,...,...,...
138,sumario_mensal_resultado,NaN,NaN,NaN
139,usina_cotista_ceg_distribuidor,NaN,NaN,NaN
140,usina_cotista_liquidacao_parcela,NaN,NaN,NaN
141,usina_cotista_nuclear,NaN,NaN,NaN


In [ ]:
!pip install requests
!pip install mercury 


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ggomessa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ggomessa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
import json
import yaml
import requests
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple, Union
import shutil
from deepdiff import DeepDiff

class MetadataMonitor:
    def __init__(self):
        # Define a URL base da API para buscar dados de datasets específicos
        self.api_base_url = 'https://dadosabertos.ccee.org.br/api/3/action/package_show?id='
        
        # Define os diretórios onde os arquivos de metadados serão armazenados
        self.source_dir = Path('source_metadata')  # Diretório para os metadados originais
        self.changes_dir = Path('changes')          # Diretório para armazenar mudanças detectadas
        self.metadata_dir = Path('metadata')        # Diretório para metadados atualizados
        
        # Garantir que todos os diretórios necessários existam, criando-os se necessário
        for directory in [self.source_dir, self.changes_dir, self.metadata_dir]:
            directory.mkdir(exist_ok=True)  # Este método não gera erro se o diretório já existir

    def fetch_api_data(self, dataset_id: str) -> Union[Dict, None]:
        """Busca dados da API com tratamento de erros.

        Este método realiza uma requisição GET para a API, utilizando o ID do dataset.
        Se a requisição falhar por qualquer razão (tempo limite, servidor fora do ar, etc.),
        o erro será tratado e uma mensagem será exibida. Retorna os dados em formato JSON
        ou None em caso de erro.
        """
        try:
            response = requests.get(f"{self.api_base_url}{dataset_id}")
            response.raise_for_status()  # Lança um erro se a resposta for um código de erro HTTP
            return response.json()  # Retorna a resposta JSON caso a requisição tenha sido bem-sucedida
        except requests.exceptions.RequestException as e:
            print(f"Erro ao buscar dados para {dataset_id}: {str(e)}")
            return None  # Retorna None se a requisição falhar

    def load_json_file(self, file_path: Path) -> Union[Dict, None]:
        """Carrega um arquivo JSON com tratamento de erros.

        Este método tenta abrir e ler um arquivo JSON. Se ocorrer um erro (por exemplo, o arquivo não existe
        ou não é um JSON válido), uma mensagem de erro será exibida e None será retornado.
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)  # Retorna o conteúdo do arquivo JSON como um dicionário
        except Exception as e:
            print(f"Erro ao carregar o arquivo JSON {file_path}: {str(e)}")
            return None  # Retorna None em caso de erro

    def save_json_file(self, data: Dict, file_path: Path) -> bool:
        """Salva um arquivo JSON com tratamento de erros.

        Este método recebe um dicionário e o salva no caminho especificado como um arquivo JSON.
        Se a operação falhar (por exemplo, problemas de permissão ou espaço em disco),
        uma mensagem de erro será exibida e False será retornado.
        """
        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)  # Salva o dicionário como JSON formatado
            return True  # Retorna True se o arquivo foi salvo com sucesso
        except Exception as e:
            print(f"Erro ao salvar o arquivo JSON {file_path}: {str(e)}")
            return False  # Retorna False em caso de erro

    def load_yaml_file(self, file_path: Path) -> Union[Dict, None]:
        """Carrega um arquivo YAML com tratamento de erros.

        Semelhante ao método load_json_file, este método abre um arquivo YAML e tenta carregá-lo
        em um dicionário Python. O tratamento de erros é realizado para garantir que qualquer
        problema com a leitura do arquivo seja relatado.
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f)  # Utiliza safe_load para evitar problemas de segurança
        except Exception as e:
            print(f"Erro ao carregar o arquivo YAML {file_path}: {str(e)}")
            return None  # Retorna None em caso de erro

    def save_yaml_file(self, data: Dict, file_path: Path) -> bool:
        """Salva um arquivo YAML com tratamento de erros.

        Recebe um dicionário e o salva em formato YAML no caminho fornecido. Caso ocorra um erro
        durante a operação, como problemas de permissão, uma mensagem de erro será exibida
        e False será retornado.
        """
        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                yaml.dump(data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)  # Salva o dicionário como YAML
            return True  # Indica que o arquivo foi salvo com sucesso
        except Exception as e:
            print(f"Erro ao salvar o arquivo YAML {file_path}: {str(e)}")
            return False  # Retorna False em caso de erro

    def compare_json_data(self, old_data: Dict, new_data: Dict) -> bool:
        """Compara dados JSON usando DeepDiff.

        Utiliza a biblioteca DeepDiff para identificar diferenças entre dois dicionários JSON.
        Retorna True se houver alterações e False se os dados forem idênticos, ignorando
        a ordem das chaves.
        """
        diff = DeepDiff(old_data, new_data, ignore_order=True)  # Compara os dicionários
        return bool(diff)  # Retorna True se houver diferenças

    def update_yaml_metadata(self, yaml_data: Dict, new_source_data: Dict) -> Dict:
        """Atualiza os metadados YAML com informações de nova versão e fonte.

        Este método atualiza a versão e as fontes no dicionário de metadados YAML. Ele gera
        um novo número de versão baseado na versão atual e adiciona informações como a nova
        fonte, se ainda não estiver presente. Também atualiza o timestamp da última atualização.
        """
        # Obtém o número da versão atual, considerando que a versão é armazenada no formato 'id_x'
        current_version = max(int(v.get('id_1', '0').split('_')[1]) 
                            for v in yaml_data.get('version', [{'id_1': 'id_0'}]))
        
        # Adiciona uma nova versão ao histórico de versões
        new_version = {'id_' + str(current_version + 1): datetime.now().isoformat()}  # Criação do novo ID de versão
        yaml_data['version'].append(new_version)  # Adiciona a nova versão à lista de versões
        
        # Cria um dicionário com os detalhes da nova fonte
        new_source = {
            'created': new_source_data.get('created'),  # Data de criação da nova fonte
            'id': new_source_data.get('id'),            # ID da nova fonte
            'name': new_source_data.get('name')         # Nome da nova fonte
        }
        
        # Verifica se a nova fonte já existe no YAML
        source_exists = any(
            source.get('id') == new_source['id']
            for source in yaml_data.get('source', [])  # Itera sobre fontes existentes
        )
        
        if not source_exists:
            yaml_data['source'].append(new_source)  # Adiciona a nova fonte se não estiver presente
        
        # Atualiza o timestamp de última atualização para a hora atual
        yaml_data['last_update'] = datetime.now().isoformat()
        
        return yaml_data  # Retorna o dicionário de metadados atualizado

    def process_dataset(self, json_file: Path) -> None:
        """Processa um arquivo de dataset único.

        Este método realiza todo o fluxo de processamento de um dataset específico. Ele começa
        carregando os dados antigos, busca novas informações na API, compara as versões e, se
        alterações forem detectadas, atualiza os metadados e grava as mudanças. Se não forem
        encontradas alterações, informa que nenhum update é necessário.
        """
        dataset_id = json_file.stem  # Obtém o ID do dataset a partir do nome do arquivo
        print(f"\nProcessando dataset: {dataset_id}")
        
        # Carrega o JSON existente do arquivo
        old_data = self.load_json_file(json_file)
        if not old_data:
            return  # Se não for possível carregar os dados antigos, encerra o processamento

        # Busca novos dados da API usando o ID do dataset
        new_data = self.fetch_api_data(dataset_id)
        if not new_data:
            return  # Se a busca falhar, encerra o processamento

        # Compara os dados antigos com os novos
        if self.compare_json_data(old_data, new_data):
            print(f"Alterações detectadas em {dataset_id}")
            
            # Salva dados alterados no diretório de mudanças, criando um nome único baseado no timestamp atual
            change_file = self.changes_dir / f"{dataset_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            if self.save_json_file(new_data, change_file):
                print(f"Alterações salvas em {change_file}")
            
            # Atualiza os metadados YAML correspondentes ao dataset
            yaml_file = self.metadata_dir / f"{dataset_id}.yaml"
            if yaml_file.exists():
                yaml_data = self.load_yaml_file(yaml_file)
                if yaml_data:
                    updated_yaml = self.update_yaml_metadata(yaml_data, new_data['result']['resources'][0])
                    if self.save_yaml_file(updated_yaml, yaml_file):
                        print(f"Metadados YAML atualizados para {dataset_id}")
            
            # Atualiza o JSON de fonte com os novos dados obtidos da API
            self.save_json_file(new_data, json_file)
        else:
            print(f"Nenhuma alteração detectada em {dataset_id}")  # Informa que não houve mudanças

    def run(self):
        """Método principal de execução do monitoramento de metadados.

        Este método inicia o processo geral de monitoramento ao buscar todos os arquivos JSON
        disponíveis no diretório de source_metadata e processá-los um por um. Ele fornece
        informações ao usuário sobre o total de arquivos e o progresso do processamento.
        """
        print("Iniciando o processo de monitoramento de metadados...")
        
        # Lista todos os arquivos JSON no diretório de origem
        json_files = list(self.source_dir.glob('*.json'))
        total_files = len(json_files)
        
        print(f"Encontrados {total_files} arquivos JSON para processar")
        
        # Itera sobre cada arquivo JSON encontrado e o processa
        for i, json_file in enumerate(json_files, 1):
            print(f"\nProcessando arquivo {i} de {total_files}")
            self.process_dataset(json_file)  # Chama o método que processa o dataset específico
        
        print("\nMonitoramento de metadados concluído!")  # Informa que todo o processamento foi concluído

if __name__ == "__main__":
    monitor = MetadataMonitor()  # Cria uma instância do monitor de metadados
    monitor.run()  # Chama o método run para iniciar o monitoramento


Starting metadata monitoring process...
Found 143 JSON files to process

Processing file 1 of 143

Processing dataset: rrh_repasse
No changes detected in rrh_repasse

Processing file 2 of 143

Processing dataset: penalidade_preco_mensal
No changes detected in penalidade_preco_mensal

Processing file 3 of 143

Processing dataset: sazonalizacao_mre_gf_fator_ajuste
No changes detected in sazonalizacao_mre_gf_fator_ajuste

Processing file 4 of 143

Processing dataset: contrato_montante_submercado_entrega_ccearq
No changes detected in contrato_montante_submercado_entrega_ccearq

Processing file 5 of 143

Processing dataset: leilao_acr_pch
No changes detected in leilao_acr_pch

Processing file 6 of 143

Processing dataset: encargo_horario_submercado
No changes detected in encargo_horario_submercado

Processing file 7 of 143

Processing dataset: penalidade_classe_mensal
No changes detected in penalidade_classe_mensal

Processing file 8 of 143

Processing dataset: biomassa_leilao
No changes de

In [ ]:
import requests 
import json 
import os 
from datetime 
import datetime 
import yaml 
from pathlib import Path import shutil from typing import Dict, Union import hashlib import time 
class MetadataChangeDetector:
    def __init__(self):
        """Inicializa o objeto MetadataChangeDetector.
        
        Define a URL base da API e os diretórios necessários para
        armazenar os metadados e mudanças. Garante que os diretórios
        existam, criando-os se necessário.
        """
        self.api_base_url = 'https://dadosabertos.ccee.org.br/api/3/action/package_show?id='
        self.source_metadata_dir = Path('/workspaces/automation_data_contract/Liberalizados/source_metadata')
        self.changes_dir = Path('changes')
        self.metadata_dir = Path('metadata')
        
        for directory in [self.source_metadata_dir, self.changes_dir, self.metadata_dir]:
            directory.mkdir(exist_ok=True)

    def fetch_api_data(self, dataset_id: str) -> Union[Dict, None]:
        """Busca dados da API com um mecanismo de retry.
        
        Parâmetros:
        dataset_id (str): O identificador único do conjunto de dados.

        Retorna:
        Union[Dict, None]: Um dicionário com os dados JSON da API,
                           ou None se a solicitação falhar.
        """
        max_retries = 3
        retry_delay = 2

        for attempt in range(max_retries):
            try:
                response = requests.get(f"{self.api_base_url}{dataset_id}")
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                if attempt == max_retries - 1:
                    print(f"Falha ao buscar dados para {dataset_id}: {str(e)}")
                    return None
                time.sleep(retry_delay)
        return None

    def load_json_file(self, file_path: Path) -> Union[Dict, None]:
        """Carrega e analisa um arquivo JSON.
        
        Parâmetros:
        file_path (Path): O caminho para o arquivo JSON.

        Retorna:
        Union[Dict, None]: Um dicionário com os dados JSON carregados,
                           ou None se ocorrer um erro.
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            print(f"Erro ao carregar {file_path}: {str(e)}")
            return None

    def calculate_json_hash(self, json_data: Dict) -> str:
        """Calcula o hash dos dados JSON para comparação.
        
        Parâmetros:
        json_data (Dict): Os dados JSON a serem hashados.

        Retorna:
        str: O hash SHA-256 dos dados JSON como uma string.
        """
        return hashlib.sha256(
            json.dumps(json_data, sort_keys=True).encode()
        ).hexdigest()

    def update_yaml_metadata(self, yaml_path: Path, new_resource: Dict):
        """Atualiza o arquivo YAML com nova versão e informações da fonte.
        
        Parâmetros:
        yaml_path (Path): O caminho para o arquivo YAML a ser atualizado.
        new_resource (Dict): As novas informações do recurso a serem 
                             adicionadas ao YAML.

        Retorna:
        None: Imprime o resultado da operação.
        """
        try:
            with open(yaml_path, 'r', encoding='utf-8') as f:
                yaml_data = yaml.safe_load(f)

            current_versions = yaml_data.get('version', [])
            max_id = 0
            for version in current_versions:
                for key in version:
                    if key.startswith('id_'):
                        try:
                            id_num = int(key.split('_')[1])
                            max_id = max(max_id, id_num)
                        except ValueError:
                            continue

            new_version = {f'id_{max_id + 1}': datetime.now().isoformat()}
            yaml_data['version'].append(new_version)

            if 'source' not in yaml_data:
                yaml_data['source'] = []
            yaml_data['source'].append(new_resource)

            yaml_data['last_update'] = datetime.now().isoformat()

            with open(yaml_path, 'w', encoding='utf-8') as f:
                yaml.dump(yaml_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)

            print(f"Arquivo YAML atualizado: {yaml_path}")
            return True
        except Exception as e:
            print(f"Erro ao atualizar YAML {yaml_path}: {str(e)}")
            return False

    def process_metadata_changes(self):
        """Função principal para processar e detectar mudanças nos metadados.
        
        Percorre todos os arquivos JSON no diretório de metadados de origem,
        carrega os dados existentes, busca dados atuais da API e compara as
        mudanças. Atualiza os arquivos YAML e registra mudanças detectadas.

        Retorna:
        None: Imprime um resumo das operações ao final.
        """
        processed = 0
        changes_detected = 0
        errors = 0

        for json_file in self.source_metadata_dir.glob('*.json'):
            try:
                dataset_id = json_file.stem
                print(f"\nProcessando {dataset_id}...")

                existing_data = self.load_json_file(json_file)
                if not existing_data:
                    errors += 1
                    continue

                current_data = self.fetch_api_data(dataset_id)
                if not current_data:
                    errors += 1
                    continue

                existing_hash = self.calculate_json_hash(existing_data)
                current_hash = self.calculate_json_hash(current_data)

                if existing_hash != current_hash:
                    print(f"Mudanças detectadas para {dataset_id}")
                    changes_detected += 1

                    change_file = self.changes_dir / f"{dataset_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                    with open(change_file, 'w', encoding='utf-8') as f:
                        json.dump(current_data, f, ensure_ascii=False, indent=4)

                    new_resource = {
                        'created': datetime.now().isoformat(),
                        'id': current_data['result']['resources'][0]['id'],
                        'name': current_data['result']['resources'][0]['name']
                    }

                    yaml_path = self.metadata_dir / f"{dataset_id}.yaml"
                    if yaml_path.exists():
                        self.update_yaml_metadata(yaml_path, new_resource)

                    with open(json_file, 'w', encoding='utf-8') as f:
                        json.dump(current_data, f, ensure_ascii=False, indent=4)

                processed += 1

            except Exception as e:
                print(f"Erro ao processar {dataset_id}: {str(e)}")
                errors += 1

        print("\nProcessamento Completo!")
        print(f"Total de arquivos processados: {processed}")
        print(f"Mudanças detectadas: {changes_detected}")
        print(f"Erros encontrados: {errors}")

if __name__ == "__main__":
    """Ponto de entrada para a execução do script.

    Cria uma instância da classe MetadataChangeDetector e inicia o
    processamento das mudanças de metadados.
    """
    detector = MetadataChangeDetector()
    detector.process_metadata_changes()



Processing rrh_repasse...



Processing penalidade_preco_mensal...

Processing sazonalizacao_mre_gf_fator_ajuste...

Processing contrato_montante_submercado_entrega_ccearq...

Processing leilao_acr_pch...

Processing encargo_horario_submercado...

Processing penalidade_classe_mensal...

Processing biomassa_leilao...

Processing energia_incentivada_mensal...

Processing parcela_usina_montante_mensal...

Processing sazonalizacao_contrato_ccearq...

Processing contrato_montante_mod_proinfa_agente...

Processing fotovoltaica_gf_definida...

Processing leilao_acr_eol...

Processing sazonalizacao_ccgf...

Processing sumario_distrib_mensal...

Processing rd_atend_prod_previo_horario...

Processing energia_reserva_consumo_referencia...

Processing ccc_ctg_beneficiario...

Processing rd_encargos_contab_mensal...

Processing biomassa_acl...

Processing consumo_horario_perfil_agente...

Processing rrh_geracao_submercado_horaria...

Processing mcsd_cedente_cessionario...

Processing biomassa_proinfa...

Processing perda_rb_h

In [4]:
import os
import requests
import json

# Define the base URL for the API
api_base_url = 'https://dadosabertos.ccee.org.br/api/3/action/package_show?id='
package_list_url = 'https://dadosabertos.ccee.org.br/api/3/action/package_list'

# Create the 'source_metadata' directory if it doesn't exist
output_dir = 'source_metadata'
os.makedirs(output_dir, exist_ok=True)

# Step 1: Make a GET request to fetch the list of package IDs
response = requests.get(package_list_url)

# Check if the response was successful
if response.status_code == 200:
    # Get the response data
    response_data = response.json()
    
    # Ensure 'result' key exists and is iterable
    if 'result' in response_data and isinstance(response_data['result'], list):
        # Step 2: Iterate over each package ID in the result
        for index, package_id in enumerate(response_data['result']):
            # Construct the complete API URL for the current package
            api_url = f"{api_base_url}{package_id}"
            
            # Make a GET request to fetch the package metadata
            response = requests.get(api_url)

            # Check if the response for the package metadata was successful
            if response.status_code == 200:
                # Get the metadata response data
                package_metadata = response.json()

                # Step 3: Save the metadata in a JSON file named after the package ID
                output_file = os.path.join(output_dir, f'{package_id}.json')
                
                with open(output_file, 'w', encoding='utf-8') as json_file:
                    json.dump(package_metadata, json_file, ensure_ascii=False, indent=4)

                print(f"Response for package ID '{package_id}' has been saved to {output_file}")
            else:
                print(f"Failed to retrieve data from API for package ID '{package_id}'. Status code: {response.status_code}")
    else:
        print("No results found in the package list.")
else:
    print(f"Failed to retrieve the package list. Status code: {response.status_code}")


Response for package ID 'acl_eol' has been saved to source_metadata/acl_eol.json
Response for package ID 'agente_qtd_contabilizacao' has been saved to source_metadata/agente_qtd_contabilizacao.json
Response for package ID 'alivio_penalidade_distrib_mensal' has been saved to source_metadata/alivio_penalidade_distrib_mensal.json
Response for package ID 'biomassa_acl' has been saved to source_metadata/biomassa_acl.json
Response for package ID 'biomassa_fonte_geracao' has been saved to source_metadata/biomassa_fonte_geracao.json
Response for package ID 'biomassa_geracao_acr_acl_modalidade' has been saved to source_metadata/biomassa_geracao_acr_acl_modalidade.json
Response for package ID 'biomassa_gf_definida' has been saved to source_metadata/biomassa_gf_definida.json
Response for package ID 'biomassa_leilao' has been saved to source_metadata/biomassa_leilao.json
Response for package ID 'biomassa_proinfa' has been saved to source_metadata/biomassa_proinfa.json
Response for package ID 'biom

In [ ]:
import json
import yaml
from datetime import datetime
import os
from pathlib import Path

def process_json_to_yaml(json_data):
    """
    Processa os dados em JSON e os converte em um formato estruturado de YAML.

    Parâmetros:
    json_data (dict): O dicionário de dados JSON contendo metadados 'result' com chaves como 'name', 'notes', 'author', etc.

    Retorna:
    dict: Um novo dicionário estruturado para saída em YAML, incluindo metadados, informações de recursos, extras e campos.
    """
    # Extrai informações básicas
    result = json_data['result']
    metadata = {
        'name': result['name'],
        'description': result['notes'],
        'owner': result['author'],
        'version': [
            {'1': datetime.now().isoformat()}
        ],
        'source': [],
        'last_update': datetime.now()
    }
    
    # Processa os recursos
    for resource in result['resources']:
        source_item = {
            'created': resource['created'],
            'id': resource['id'],
            'name': resource['name']
        }
        metadata['source'].append(source_item)
    
    # Extrai extras
    extras_dict = {extra['key']: extra['value'] for extra in result['extras']}
    metadata['periodice'] = extras_dict.get('Frequência de Atualização', '')
    
    # Processa tags
    tags = [tag['display_name'] for tag in result['tags']]
    
    # Adiciona estrutura de campos
    metadata['fields'] = {
        'field1': {
            'type': 'datetime',
            'description': '',
            'label': '',
            'null': True,
            'business_rule': ''
        },
    }
    
    # Adiciona informações de sincronização
    metadata['sync'] = 's3://xxxxxxxxxxxxxxxx/xxxxxxxxxxx'
    
    return metadata

def save_yaml_file(data, filename):
    """
    Salva um dicionário fornecido como um arquivo YAML.

    Parâmetros:
    data (dict): Os dados a serem gravados no arquivo YAML.
    filename (str): O nome do arquivo a ser salvo (sem extensão).

    Nota: O arquivo YAML de saída será salvo em um diretório predefinido.
    """
    output_directory = '/workspaces/automation_data_contract/Liberalizados/contracts'

    # Salva o arquivo YAML
    yaml_path = os.path.join(output_directory, f"{filename}.yaml")
    with open(yaml_path, 'w', encoding='utf-8') as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False, allow_unicode=True, sort_keys=False)

def process_file(json_path):
    """
    Lê um arquivo JSON, processa seu conteúdo para extrair metadados e salva como um arquivo YAML.

    Parâmetros:
    json_path (Path): O caminho para o arquivo JSON a ser processado.

    Retorna:
    bool: Verdadeiro se o processamento foi bem-sucedido, falso caso contrário.

    Exceções:
    Imprime uma mensagem de erro se o arquivo não puder ser processado.
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)  # Carrega os dados JSON do arquivo
            yaml_data = process_json_to_yaml(json_data)  # Converte os dados JSON para o formato YAML
            filename = json_data['result']['name']  # Extrai o nome para o nome do arquivo de saída
            save_yaml_file(yaml_data, filename)  # Salva no arquivo YAML
            print(f"Processed {json_path.name} -> metadata/{filename}.yaml")  # Registra o processamento bem-sucedido
            return True
    except Exception as e:
        print(f"Error processing {json_path.name}: {str(e)}")  # Registra qualquer erro encontrado durante o processamento
        return False

def main():
    """
    Função principal para orquestrar o processamento de arquivos JSON do diretório de entrada.

    Esta função busca por todos os arquivos JSON no diretório de entrada especificado, processa-os,
    e gera saídas em YAML. Ela também rastreia o número de processos bem-sucedidos e falhados.
    """
    input_directory = '/workspaces/automation_data_contract/Liberalizados/source_metadata'   
    
    # Obtém todos os arquivos JSON no diretório de origem
    source_dir = Path(input_directory)
    json_files = list(source_dir.glob('*.json'))
    
    if not json_files:
        print("No JSON files found in the source directory.")  # Registra se nenhum arquivo JSON existe
        return
    
    # Processa cada arquivo JSON
    successful = 0
    failed = 0
    
    for json_path in json_files:
        if process_file(json_path):  # Tenta processar cada arquivo JSON
            successful += 1
        else:
            failed += 1
    
    # Imprime o resumo dos resultados do processamento
    print("\nProcessing Complete!")
    print(f"Total files processed: {len(json_files)}")  # Registra o número total de arquivos processados
    print(f"Successful: {successful}")  # Registra a contagem de arquivos processados com sucesso
    print(f"Failed: {failed}")  # Registra a contagem de tentativas de processamento falhadas

if __name__ == "__main__":
    main()


Processed rrh_repasse.json -> metadata/rrh_repasse.yaml
Processed penalidade_preco_mensal.json -> metadata/penalidade_preco_mensal.yaml
Processed sazonalizacao_mre_gf_fator_ajuste.json -> metadata/sazonalizacao_mre_gf_fator_ajuste.yaml
Processed contrato_montante_submercado_entrega_ccearq.json -> metadata/contrato_montante_submercado_entrega_ccearq.yaml
Processed leilao_acr_pch.json -> metadata/leilao_acr_pch.yaml
Processed encargo_horario_submercado.json -> metadata/encargo_horario_submercado.yaml
Processed penalidade_classe_mensal.json -> metadata/penalidade_classe_mensal.yaml
Processed biomassa_leilao.json -> metadata/biomassa_leilao.yaml
Processed energia_incentivada_mensal.json -> metadata/energia_incentivada_mensal.yaml
Processed parcela_usina_montante_mensal.json -> metadata/parcela_usina_montante_mensal.yaml
Processed sazonalizacao_contrato_ccearq.json -> metadata/sazonalizacao_contrato_ccearq.yaml
Processed contrato_montante_mod_proinfa_agente.json -> metadata/contrato_montant

In [6]:
import yaml
from pathlib import Path
import requests
import time
from typing import Dict, List
import json

def load_yaml(file_path: Path) -> Dict:
    """Load YAML file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

def save_yaml(file_path: Path, data: Dict):
    """Save YAML file."""
    with open(file_path, 'w', encoding='utf-8') as f:
        yaml.dump(data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)

def fetch_api_data(resource_id: str) -> Dict:
    """Fetch data from the API with error handling and retries."""
    url = f"https://dadosabertos.ccee.org.br/api/3/action/datastore_search?resource_id={resource_id}&limit=0"
    max_retries = 3
    retry_delay = 2  # seconds
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            if attempt == max_retries - 1:
                print(f"Failed to fetch data for resource_id {resource_id}: {str(e)}")
                return None
            time.sleep(retry_delay)
    
    return None

def process_fields(api_data: Dict) -> Dict:
    """Process API response and extract field information."""
    if not api_data or 'result' not in api_data or 'fields' not in api_data['result']:
        return {}
    
    fields = {}
    # Skip the first field (usually _id) by starting from index 1
    for field in api_data['result']['fields'][1:]:
        field_id = field.get('id', '')
        info = field.get('info', {})
        
        fields[field_id] = {
            'type': field.get('type', ''),
            'description': info.get('notes', ''),
            'label': info.get('label', ''),
            'business_rule': ''
        }
    
    return fields

def update_yaml_files():
    """Main function to process all YAML files in the metadata directory."""
    meta_dir = Path('/workspaces/automation_data_contract/Liberalizados/contracts')
    if not meta_dir.exists():
        print("Metadata directory not found!")
        return
    
    yaml_files = list(meta_dir.glob('*.yaml'))
    if not yaml_files:
        print("No YAML files found in metadata directory!")
        return
    
    processed = 0
    failed = 0
    
    for yaml_path in yaml_files:
        try:
            print(f"\nProcessing {yaml_path.name}...")
            
            # Load YAML
            yaml_data = load_yaml(yaml_path)
            
            # Process each source
            new_fields = {}
            for source in yaml_data.get('source', []):
                source_id = source.get('id')
                if not source_id:
                    continue
                
                print(f"Fetching data for resource_id: {source_id}")
                api_data = fetch_api_data(source_id)
                
                if api_data:
                    fields = process_fields(api_data)
                    new_fields.update(fields)
            
            # Update fields in YAML
            if new_fields:
                yaml_data['fields'] = new_fields
                save_yaml(yaml_path, yaml_data)
                print(f"Updated fields in {yaml_path.name}")
                processed += 1
            else:
                print(f"No fields data found for {yaml_path.name}")
                failed += 1
                
        except Exception as e:
            print(f"Error processing {yaml_path.name}: {str(e)}")
            failed += 1
    
    # Print summary
    print("\nProcessing Complete!")
    print(f"Total files processed: {len(yaml_files)}")
    print(f"Successful: {processed}")
    print(f"Failed: {failed}")

if __name__ == "__main__":
    update_yaml_files()


Processing rrh_agente.yaml...
Fetching data for resource_id: 20324f6c-d7f6-4baf-841b-e50bd04140bc


Updated fields in rrh_agente.yaml

Processing geracao_consolidada.yaml...
Fetching data for resource_id: 51b5976b-7cd5-4e87-9cbf-d8f105b80c77
Fetching data for resource_id: 77cfee8f-a917-4679-b3f3-7506db4db8b2
Updated fields in geracao_consolidada.yaml

Processing ccear_resultado_mensal.yaml...
Fetching data for resource_id: fce61054-9983-40e0-8df0-662daf7f5ba7
Fetching data for resource_id: 9d7e3430-fd33-454e-a2c4-bc1b69d9cc48
Updated fields in ccear_resultado_mensal.yaml

Processing ccear_balanco_energetico_mensal.yaml...
Fetching data for resource_id: 773d3213-fa64-41c0-9dc6-29da523da274
Fetching data for resource_id: a3c3f5e1-ea9e-4255-b90b-c276557a001e
Updated fields in ccear_balanco_energetico_mensal.yaml

Processing geracao_horaria.yaml...
Fetching data for resource_id: c13f034c-f5e5-4fec-8b8a-68e66500b86f
Fetching data for resource_id: d74e5488-bcf4-47a4-b9fd-d7dbfd9fc4e4
Updated fields in geracao_horaria.yaml

Processing perda_rb_horario.yaml...
Fetching data for resource_id: 

In [7]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
import yaml
from pathlib import Path
import requests
from typing import Dict, List
import json
from datetime import datetime

class SparkTableCreator:
    def __init__(self, json_output_dir: str = '/path/to/json/output'):
        self.spark = SparkSession.builder \
            .appName("YAML to JSON") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0") \
            .getOrCreate()
        
        self.api_base_url = 'https://dadosabertos.ccee.org.br/api/3/action/datastore_search?resource_id='
        self.type_mapping = {
            'text': StringType(),
            'numeric': DoubleType(),
            'int': IntegerType(),
            'timestamp': TimestampType(),
            'date': TimestampType()
        }
        self.json_output_dir = json_output_dir

    def load_yaml(self, yaml_path: Path) -> Dict:
        """Load YAML file."""
        with open(yaml_path, 'r', encoding='utf-8') as f:
            return yaml.safe_load(f)

    def fetch_api_data(self, resource_id: str) -> Dict:
        """Fetch data from API with error handling and retries."""
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.get(f"{self.api_base_url}{resource_id}")
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                if attempt == max_retries - 1:
                    print(f"Failed to fetch data for resource_id {resource_id}: {str(e)}")
                    return None
        return None

    def create_schema(self, fields: Dict) -> StructType:
        """Create Spark schema from YAML fields, ensuring field names are uppercase."""
        schema_fields = []
        
        for field_name, field_info in fields.items():
            spark_type = self.type_mapping.get(field_info['type'].lower(), StringType())
            schema_fields.append(StructField(field_name, spark_type, True))  
        
        return StructType(schema_fields)

    def preprocess_data(self, data: List[Dict], schema: StructType) -> List[Dict]:
        """Preprocess Data for DataFrame creation with uppercase field names."""
        processed_data = []
        
        for record in data:
            processed_record = {}
            for field in schema.fieldNames():
                value = record.get(field)
                expected_type = schema[field].dataType
                
                if isinstance(value, (int, float)):
                    if expected_type == DoubleType() and isinstance(value, int):
                        processed_record[field] = float(value)
                    else:
                        processed_record[field] = value
                else:
                    processed_record[field] = value  

            processed_data.append(processed_record)
        
        return processed_data

    def create_json(self, data: List[Dict], json_file_name: str):
        """Save DataFrame as JSON."""
        try:
            processed_data = self.preprocess_data(data, self.create_schema(data[0].keys()))  # Generate schema
            df = self.spark.createDataFrame(processed_data)
            
            json_output_path = Path(self.json_output_dir) / f"{json_file_name}.json"
            df.write.mode("overwrite").json(str(json_output_path))

            print(f"Successfully saved JSON to: {json_output_path}")
            print(f"Row count: {df.count()}")
            return True
        except Exception as e:
            print(f"Error saving JSON {json_file_name}: {str(e)}")
            return False

    def process_yaml(self, yaml_path: Path):
        """Process single YAML file and create corresponding JSON files."""
        print(f"\nProcessing YAML: {yaml_path}")
        
        yaml_data = self.load_yaml(yaml_path)
        if not yaml_data:
            return
        
        schema = self.create_schema(yaml_data['fields'])
        
        for source in yaml_data['source']:
            source_id = source['id']
            json_file_name = f"{yaml_data['name']}_{source['name']}"

            print(f"\nProcessing source: {source['name']}")

            api_data = self.fetch_api_data(source_id)
            if not api_data or 'result' not in api_data or 'records' not in api_data['result']:
                continue

            # Save JSON instead of creating table
            self.create_json(
                data=api_data['result']['records'],
                json_file_name=json_file_name
            )

    def process_all_yamls(self):
        """Process all YAML files in metadata directory."""
        metadata_dir = Path('/workspaces/automation_data_contract/Liberalizados/contracts')
        if not metadata_dir.exists():
            print("Metadata directory not found!")
            return
        
        yaml_files = list(metadata_dir.glob('*.yaml'))
        if not yaml_files:
            print("No YAML files found in metadata directory!")
            return
        
        print(f"Found {len(yaml_files)} YAML files to process")
        
        for yaml_path in yaml_files:
            try:
                self.process_yaml(yaml_path)
            except Exception as e:
                print(f"Error processing {yaml_path}: {str(e)}")

def main():
    print(f"Starting JSON creation process at {datetime.now()}")
    
    # Create and run processor with the output directory for JSON files
    processor = SparkTableCreator(json_output_dir='/raw')
    processor.process_all_yamls()
    
    print(f"\nProcess completed at {datetime.now()}")

if __name__ == "__main__":
    main()


Starting JSON creation process at 2024-11-21 20:32:56.710116


24/11/21 20:32:58 WARN Utils: Your hostname, codespaces-c653a6 resolves to a loopback address: 127.0.0.1; using 10.0.4.127 instead (on interface eth0)
24/11/21 20:32:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-07e17f01-5b99-46ed-b12a-cde5e3e3203d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 277ms :: artifacts dl 12ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------

Found 143 YAML files to process

Processing YAML: /workspaces/automation_data_contract/Liberalizados/contracts/rrh_agente.yaml

Processing source: rrh_agente_2024
Error saving JSON rrh_agente_rrh_agente_2024: 'dict_keys' object has no attribute 'items'

Processing YAML: /workspaces/automation_data_contract/Liberalizados/contracts/geracao_consolidada.yaml

Processing source: geracao_consolidada_2023
Error saving JSON geracao_consolidada_geracao_consolidada_2023: 'dict_keys' object has no attribute 'items'

Processing source: geracao_consolidada_2024
Error saving JSON geracao_consolidada_geracao_consolidada_2024: 'dict_keys' object has no attribute 'items'

Processing YAML: /workspaces/automation_data_contract/Liberalizados/contracts/ccear_resultado_mensal.yaml

Processing source: ccear_resultado_mensal_2023
Error saving JSON ccear_resultado_mensal_ccear_resultado_mensal_2023: 'dict_keys' object has no attribute 'items'

Processing source: ccear_resultado_mensal_2024
Error saving JSON cc

24/11/21 20:33:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Error saving JSON contrato_montante_periodo_contrato_montante_periodo_2023: 'dict_keys' object has no attribute 'items'

Processing source: contrato_montante_periodo_2024
Error saving JSON contrato_montante_periodo_contrato_montante_periodo_2024: 'dict_keys' object has no attribute 'items'

Processing YAML: /workspaces/automation_data_contract/Liberalizados/contracts/ccgf_balanco_energetico_mensal.yaml

Processing source: ccgf_balanco_energetico_mensal_2023
Error saving JSON ccgf_balanco_energetico_mensal_ccgf_balanco_energetico_mensal_2023: 'dict_keys' object has no attribute 'items'

Processing source: ccgf_balanco_energetico_mensal_2024
Error saving JSON ccgf_balanco_energetico_mensal_ccgf_balanco_energetico_mensal_2024: 'dict_keys' object has no attribute 'items'

Processing YAML: /workspaces/automation_data_contract/Liberalizados/contracts/energia_reserva_mensal_leilao.yaml

Processing source: energia_reserva_mensal_leilao_2023
Error saving JSON energia_reserva_mensal_leilao_energ

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Count Tables") \
    .getOrCreate()


24/11/18 21:30:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
# Listar tabelas
tables = spark.sql("SHOW TABLES")

# Contar tabelas
table_count = tables.count()
print(f"Número de tabelas no spark-warehouse: {table_count/2}")


Número de tabelas no spark-warehouse: 175.5


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
import yaml
from pathlib import Path
import requests
from typing import Dict, List
from datetime import datetime

class SparkTableCreator:
    def __init__(self):
        self.spark = SparkSession.builder \
            .appName("YAML to Tables") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0") \
            .getOrCreate()
        
        self.api_base_url = 'https://dadosabertos.ccee.org.br/api/3/action/datastore_search?resource_id='
        self.type_mapping = {
            'text': StringType(),
            'numeric': DoubleType(),
            'int': IntegerType(),
            'timestamp': TimestampType(),
            'date': TimestampType()
        }

    def load_yaml(self, yaml_path: Path) -> Dict:
        """Load YAML file."""
        with open(yaml_path, 'r', encoding='utf-8') as f:
            return yaml.safe_load(f)

    def fetch_api_data(self, resource_id: str) -> Dict:
        """Fetch data from API with error handling and retries."""
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.get(f"{self.api_base_url}{resource_id}")
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                if attempt == max_retries - 1:
                    print(f"Failed to fetch data for resource_id {resource_id}: {str(e)}")
                    return None
        return None

    def create_schema(self, fields: Dict) -> StructType:
        """Create Spark schema from YAML fields, ensuring field names are uppercase."""
        schema_fields = []
        
        for field_name, field_info in fields.items():
            # Get Spark type from mapping, default to StringType
            spark_type = self.type_mapping.get(field_info['type'].lower(), StringType())
            
            # Create schema field with the original (uppercase) field name
            schema_fields.append(
                StructField(
                    field_name.upper(),  # Use uppercase field names for consistency
                    spark_type,
                    True  # nullable
                )
            )
        
        return StructType(schema_fields)

    def preprocess_data(self, data: List[Dict], schema: StructType) -> List[Dict]:
        """Preprocess Data for DataFrame creation with uppercase field names."""
        processed_data = []
        
        for record in data:
            processed_record = {}
            for field in schema.fieldNames():
                value = record.get(field)  # Match uppercase field names
                expected_type = schema[field].dataType
                
                # Check type and convert if necessary
                if isinstance(value, (int, float)):
                    if expected_type == DoubleType() and isinstance(value, int):
                        processed_record[field] = float(value)  # Convert int to float
                    else:
                        processed_record[field] = value
                else:
                    processed_record[field] = value  # Accept None or other values

            processed_data.append(processed_record)
        
        return processed_data

    def create_table(self, data: List[Dict], schema: StructType, table_name: str):
        """Create Spark table from API data with the specified name."""
        try:
            # Preprocess data to handle types properly
            processed_data = self.preprocess_data(data, schema)
            
            # Create DataFrame
            df = self.spark.createDataFrame(processed_data, schema)
            
            # Register as temp table
            df.createOrReplaceTempView(table_name)
            
            # Save as a permanent table
            df.write.mode("overwrite").saveAsTable(table_name)
            
            print(f"Successfully created table: {table_name}")
            print(f"Row count: {df.count()}")
            
            return True
        except Exception as e:
            print(f"Error creating table {table_name}: {str(e)}")
            return False

    def process_yaml(self, yaml_path: Path):
        """Process single YAML file and create corresponding tables."""
        print(f"\nProcessing YAML: {yaml_path}")
        
        # Load YAML
        yaml_data = self.load_yaml(yaml_path)
        if not yaml_data:
            return
        
        # Create schema from fields
        schema = self.create_schema(yaml_data['fields'])
        
        # Process each source
        for source in yaml_data['source']:
            source_id = source['id']
            source_name = source['name']  # Use the name from the source for table creation
            
            # Create a table name based on the source name
            table_name = source_name  # Directly use the source name
            
            print(f"\nProcessing source: {source_name}")
            
            # Fetch data from API
            api_data = self.fetch_api_data(source_id)
            if not api_data or 'result' not in api_data or 'records' not in api_data['result']:
                continue
            
            # Create table with the specified source name
            self.create_table(
                data=api_data['result']['records'],
                schema=schema,
                table_name=table_name
            )

    def process_all_yamls(self):
        """Process all YAML files in the metadata directory."""
        metadata_dir = Path('/workspaces/automation_data_contract/Liberalizados/contracts')
        if not metadata_dir.exists():
            print("Metadata directory not found!")
            return
        
        yaml_files = list(metadata_dir.glob('*.yaml'))
        if not yaml_files:
            print("No YAML files found in metadata directory!")
            return
        
        print(f"Found {len(yaml_files)} YAML files to process")
        
        for yaml_path in yaml_files:
            try:
                self.process_yaml(yaml_path)
            except Exception as e:
                print(f"Error processing {yaml_path}: {str(e)}")

def main():
    # Set up logging
    print(f"Starting table creation process at {datetime.now()}")
    
    # Create and run processor
    processor = SparkTableCreator()
    processor.process_all_yamls()
    
    print(f"\nProcess completed at {datetime.now()}")

if __name__ == "__main__":
    main()


Starting table creation process at 2024-11-18 22:18:32.507473


24/11/18 22:18:34 WARN Utils: Your hostname, codespaces-c653a6 resolves to a loopback address: 127.0.0.1; using 10.0.5.234 instead (on interface eth0)
24/11/18 22:18:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-425c7831-f6fd-49ec-9f78-2c06b3725459;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 341ms :: artifacts dl 10ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------

Found 143 YAML files to process

Processing YAML: /workspaces/automation_data_contract/Liberalizados/contracts/rrh_agente.yaml

Processing source: rrh_agente_2024


24/11/18 22:18:48 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


KeyboardInterrupt: 